In [16]:
# 9 - 6 Optional Args in Decorator: 
from functools import wraps, partial
def d(func=None, decorator_arg="dec"):
    if func is None: 
        return partial(d, decorator_arg=decorator_arg)
    
    @wraps(func)
    def wrap(*args, **kwargs):
        print("wrapped", decorator_arg)
        func(*args, **kwargs)
    return wrap

# when there's decorator_arg in decorator, then decoration becomes d(args)(func)
# so we need a decorator without args, so we can do func = d(func). 
# partial does the trick
@d(decorator_arg="another decorator_arg")
def foo(a, b="hello"):
    # You can print the current function's name
    print(foo.__name__)
# foo = d(foo, decorator_arg="another decorator_arg")

foo(2,3)

wrapped another decorator_arg
foo


In [1]:
# 9 - 7 Type checking
# 1 simple structure
from functools import partial, wraps
def type_check(*types):
    def decorate(func):
        @wraps(func)
        def wrap(*args, **kwargs):
            return func(*args, **kwargs)
        return wrap
    return decorate

# So you can add layers on top of the decorator layer, so another way to add optional args
@type_check()
def foo(a, b="hello"):
    print(foo.__name__, "a: ", a)

foo(1)
        
# 2 type checking 
from inspect import signature 
# The mechanism is to bind function with their args signature. 
# So we can compare intended types as a partially bound signature, 
# Actual function as a fully bound signature 
def type_check(func, fully_bound_func, *desired_types):
    sig = signature(func)
    bound_types = sig.bind_partial(*desired_types).arguments
    # sig.bind.arguments = OrderedDict([(arg_name, arg_val) ... ])
    for arg_name, arg_value in fully_bound_func.arguments.items():
        if arg_name in bound_types:
            
            if not isinstance(arg_value, bound_types[arg_name]):
                print(f"Type check error, for {arg_name}, \
                      want: {bound_types[arg_name]}, got {arg_value}")


# bool will be considered an int
type_check(foo, signature(foo).bind(22.4), int)    


foo a:  1
Type check error, for a,                       want: <class 'int'>, got 22.4


In [4]:
# 9-8 Define Decorator as part of a class 
class Foo:
    def dec1(self, func):
        def wrapped(*args, **kwargs):
            print("dec1: ")
            func(*args, **kwargs)
        return wrapped
    
    @staticmethod
    def dec2(func):
        def wrapped(*args, **kwargs):
            print("dec2: ")
            func(*args, **kwargs)
        return wrapped
    
@Foo.dec2
def foo(a, b):
    print("foo")

foo(1,2)

dec2: 
foo


In [11]:
# 9-9.1 Property 
class Foo:
    @property
    def rico(self):
        try:
            return self.val
        except AttributeError:
            return "rico not set"
    @rico.setter
    def rico(self, value):
        self.val = value
    
    # A property decorator is to define a property object like this.
    another_attr = property()
    @another_attr.getter
    def another_attr(self):
        try:
            return self.another_val
        except AttributeError:
            return "another attr not set"
    
    @another_attr.setter
    def another_attr(self, value):
        self.another_val = value

f = Foo()
f.rico = "random val"
f.rico

f.another_attr


'another attr not set'

In [26]:
# 9-9 Decorator implemented as a class (Pre-cursor of descriptor)
# Profiling Layer
from functools import wraps

class Profile:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
    def __call__(self, *args, **kwargs):
        self.ncalls = self.ncalls + 1
        return self.__wrapped__(*args, **kwargs)
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            from types import MethodType
            return MethodType(self, instance)
    
@Profile
def some_func(a,b):
    return a,b

some_func(1,2)
some_func(1,2)
some_func(1,2)
# see 3
some_func.ncalls
# Profile object, since it's been wrapped.
type(some_func)

class Spam:
    spam_attr = "hello spam"
    # methods in a class are function objects, and needs to be bound to an instance of Spam. 
    # This is achieved by calling spam.__get__(s, x). A descriptor protocol
    # Since it's been wrapped as Profile object, Profile class needs __get__ as well.
    # MethodType is to bind the function with the corresponding object
    @Profile
    def spam(self, x):
        return x

s = Spam()
s.spam(2)

# To illustrate __get__ : class calls its method's __get__ to bind
def spam_outside_class(self,x):
    print(self.spam_attr)
    return x
# This creates a bound function to Spam, and calls the function.
spam_outside_class.__get__(s, Spam)(2)


hello spam


2

In [29]:
# 9 - 10 static method must wrap the decorated function, not the other way around 
def dec(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
    return wrapper

class Foo:
    @staticmethod
    @dec
    def foo():
        print(foo.__name__)
Foo.foo()

wrapped


In [41]:
# 9 - 11 
# Adding more args while not breaking the wrapped function signature
def dec(func):
    def wrapper(*args, debug = False, **kwargs):
        if debug:
            print("debugging")
        func(*args, **kwargs)
    return wrapper

@dec
def some_func(a,b):
    print(a,b)

some_func(1,2)
# We added another arg here!
some_func(3,4, debug=True)

# 9 - 12 patch a class function with class decorator
def class_dec(cls):
    original_getattribute = cls.__getattribute__
    def new_getattribute(self, name):
        print("new attribute")
        return original_getattribute(self, name)
    cls.__getattribute__ = new_getattribute
    return cls

@class_dec
class Foo:
    bar = 1
    def __init__(self) -> None:
        self.baz = 3
    def spam(self):
        print("spam")

# now can see "new attribute"
Foo().spam()
# this is a class variable, but not going through __getattribute__
# One big reason to use getattr is because we can supply a default value
print(getattr(Foo, "bar", "default string"))
print(getattr(Foo(), "baz", "default string"))

1 2
debugging
3 4
new attribute
spam
new attribute


3

In [1]:
# 9 - 13 metaclass 
# Simple example 1
from typing import Any
# metaclass requires to be child class of type. Otherwise "NoInstances() takes no arguments" in metaclass declaration
class NoInstances(type):
    # So an instance of class NoInstances, the subsequent class, cannot be instantiated as it could be seen as a callable.
    def __call__(self, *args: Any, **kwargs: Any): 
        raise TypeError("No instances pls")

class Spam(metaclass = NoInstances):
    def __init__(self) -> None:
       pass 
    
    @staticmethod
    def foo():
        print("foo")
        
    def __call__(self, *args: Any, **kwargs: Any): 
        print("yeehaw")

Spam.foo()

try:
    s = Spam()
except TypeError as e:
    print(e)

# example 2
class Singleton(type):
    def __init__(cls, *args: Any, **kwargs: Any):
        cls._instance = None

    def __call__(cls, *args: Any, **kwargs: Any): 
        if cls._instance is None:
            print("none")
            # super (class, object)
            # = super(Singleton, cls).__call__(*args, *kwargs)
            # = type(Singleton).__call__(cls,*args, **kwargs), which calls ctor of cls
            cls._instance = super().__call__(*args, **kwargs)
            
        print(f"args: {args}, kwargs: {kwargs}, cls:{cls}")
        return cls._instance

class Foo(metaclass = Singleton):
    pass

f = Foo()
f2 = Foo()
type(f)

foo
No instances pls
none
args: (), kwargs: {}, cls:<class '__main__.Foo'>
args: (), kwargs: {}, cls:<class '__main__.Foo'>


__main__.Foo

In [12]:
# 9-14 seqeuence of calling: __prepare__ -> execution of the class body, to fill the dict with the attrs  
# -> meta meta's __call__ -> __new__ -> __call__ -> __init__
class M(type):
    def __call__(mmcls, *args, **kwargs):
        print("M's call", args, kwargs)
        return super().__call__(*args, **kwargs)

class MM(type, metaclass=M):
    def __prepare__(cls, *args, **kw):
        print("MM Prepare")
        return {}
    def __new__(mcls, *args, **kw):
        print("MM __new__")
        return super().__new__(mcls, *args, **kw)
    def __call__(mmcls, *args, **kwargs):
        print("MM's call", args, kwargs)
        return super().__call__(*args, **kwargs)

class klass(metaclass=MM):
    def __init__(self) -> None:
        print(f"{self.__class__.__name__} initialized")

klass()

MM Prepare
M's call ('klass', (), {'__module__': '__main__', '__qualname__': 'klass', '__init__': <function klass.__init__ at 0x7f12641b8310>}) {}
MM __new__
MM's call () {}
klass initialized


In [35]:
# 9 - 14 Get order of initialization
from collections import OrderedDict
class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        # cls is Meta
        print("cls: ", cls, " bases: ", bases)
        return OrderedDict()
    def __new__(cls, clsname, bases, clsdict):
        # converting clsdict back to Dict.
        d = dict(clsdict)
        # adding an item in dict from prepare will actually become a new attribute in the function!
        # methods, class variables will go into clsdict (will be available at new), while regular instance attributes won't 
        d["_order"] = [k for k, _ in clsdict.items()]
        #TODO
        print(f'd: {d}')
        # Equivalent to
        # return super().__new__(cls, clsname, bases, d)
        return type.__new__(cls, clsname, bases, d)
        
class Foo(metaclass = Meta):
    lol_2 = 3
    lol = 0
    def __init__(self):
        self.lol = 1
        print(self.__class__.__name__)
Foo()._order

cls:  <class '__main__.Meta'>  bases:  ()
d: {'__module__': '__main__', '__qualname__': 'Foo', 'lol_2': 3, 'lol': 0, '__init__': <function Foo.__init__ at 0x7f127414c670>, '_order': ['__module__', '__qualname__', 'lol_2', 'lol', '__init__']}
Foo


['__module__', '__qualname__', 'lol_2', 'lol', '__init__']

In [43]:
# 9 - 15: This is to control certain behaviors when a class is created. 
# Requires all 3 functions to have the arg. 
# Another alternative is to pass in a class var in Foo (see 9 - 14), but that'd be available at new and after.
class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases, debug = False):
        if debug:
            print("__prepare__")
        return super().__prepare__(clsname, bases)

    def __new__(cls, name, bases, clsdict, debug = False):
        if debug:
            print("__new__")
        return super().__new__(cls, name, bases, clsdict)
    
    def __init__(self, name, bases, ns, debug = False):
        return super().__init__(name, bases, ns)

# Now you now here you add args for metaclass to control class creation behaviors
class Foo(metaclass = Meta, debug = True):
    # another alternative is in __new__, read the class var.
    # debug = True
    def __init__(self):
        print("Foo")

Foo()

__prepare__
__new__
Foo


In [68]:
# 9 - 16, do signature check 
# Simple example
from inspect import Signature, Parameter 
def make_sig(*args):
    params = [Parameter(name,Parameter.POSITIONAL_OR_KEYWORD) for name in args]
    return Signature(params)
def foo(*args, **kwargs):
    signature = make_sig("x", "y", "z")
    # Because we are using bind instead of bind_partial, args, kwargs must be strings EXACTLY the same as the signature.
    bounded_vals=  signature.bind(*args, **kwargs)

# Too many args
# foo(1,2,5,6)
foo(1,2,3)

# More practical example - Structure with common structure of things
class Structure:
    _fields = []
    def __init__(self, *args, **kwargs):
        __sigs__ =  make_sig(*self._fields)
        bounded_vals = __sigs__.bind(*args, **kwargs)
        for name, val in bounded_vals.arguments.items():
            setattr(self, name, val)

class Stock(Structure):
    _fields = ["x", "y"]
    
Stock(x=1,y=2)
    

In [90]:
# 9 - 18 Manually assemble a class 
class Foo:
    def __init__(self,a, b):
        self.a = a
        self.b = b

    def cost(self):
        return self.a + self.b
cls_dict = {
    "__init__": Foo.__init__, 
    "cost": Foo.cost,
}

import types
FooEquivalent = types.new_class("FooEquivalent", (), {}, lambda ns: ns.update(cls_dict))
# __module__ is used in __repr__, or pickling
FooEquivalent.__module__ = __name__
f_e = FooEquivalent(2,3)
print(f_e.cost())

# Example 2, metaclass, args to class
class FakeMeta(type):
    @classmethod
    def __prepare__(cls, clsname, bases, debug = False):
        if debug:
            print("__prepare__")
        return super().__prepare__(clsname, bases)

    def __new__(cls, name, bases, clsdict, debug = False):
        if debug:
            print("__new__")
        return super().__new__(cls, name, bases, clsdict)
    
    def __init__(self, name, bases, ns, debug = False):
        return super().__init__(name, bases, ns)
# second is the baseclasses
FooABC = types.new_class("FooABC", (), {'metaclass': FakeMeta, 'debug': True}, lambda ns: ns.update(cls_dict))
FooABC(1,2)
# You can still instantiate an abstract base class, but not if it has abstractmethod.
from abc import ABCMeta,abstractmethod
class MySequence(metaclass=ABCMeta):
    @abstractmethod
    def bar(self):
        pass
try: 
    m = MySequence()
except TypeError as e:
    print(e)


5
__prepare__
__new__
Can't instantiate abstract class MySequence with abstract methods bar


In [126]:
# 9 - 18 named tuple and equivalent 
import collections
Point = collections.namedtuple("Point", ["x", "y", "z"])
p1 = Point(x = 1, y = 2, z = 3)
# Can't set attribute to namedtuple after initialization
try: 
    p1.x = 4
except AttributeError as e:
    print(e)

import operator
import types
import sys
def named_tuple(class_name, fields):
    # populate list of attrs to class_dict: field property accessors? 
    # property turns the function into a property
    class_dict = {name: property(operator.itemgetter(n)) for n, name in enumerate(fields)}
    
    # Not sure why we need a __new__ function here? 
    # Because tuple doesn't have __init__, since it's already too late to change values there.
    # __new__ (create object)-> __init__ (initialize attrbs)
    def __new__(cls, *args):
        return tuple.__new__(cls, args)
    class_dict['__new__'] = __new__
    
    print(class_dict)
    cls = types.new_class(class_name, (tuple, ), {}, lambda ns: ns.update(class_dict))
    # the frame hack
    cls.__module__ = sys._getframe(1).f_globals['__name__']
    return cls 
PointEquivalent = named_tuple("PointEquivalent", ["x", "y", "z"])
p2=PointEquivalent(1,2,3)
print(p2.z)
# It's a tuple under the hood anyway
len(p2)

can't set attribute
{'x': <property object at 0x7f124ed239f0>, 'y': <property object at 0x7f124ed23900>, 'z': <property object at 0x7f124ed239a0>, '__new__': <function named_tuple.<locals>.__new__ at 0x7f124ee76a60>}


3

In [129]:
# itemgetter(n): operator is a built in module. 
# itemgetter(n) returns a callable that gets nth element of a list like object
f = operator.itemgetter(2)
# see 3. 
f((1,2,3))

3

In [1]:
def getter(self):
    print(self, self[0])
    return "lol"

import operator
class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        # Not sure why we don't need cls in __init__?
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
            # setattr(cls, name, property(getter))
            # This is a very advanced technique. you pass in getter into property(getter), 
            # Then when you call Point.x, it calls getter(self). 
            # self, is actually the tuple object. you can even do self[0]
            # same as operator.itemgetter(0)(self).  
            setattr(cls, name, property(operator.itemgetter(n)))


class StructTuple(tuple, metaclass=StructTupleMeta):
     _fields = []
     def __new__(cls, *args):
         if len(args) != len(cls._fields):
             raise ValueError(f"{len(cls._fields)} args required")
         return super().__new__(cls, args)

class Point(StructTuple):
    _fields = ["x", "y"]

p = Point(1,2)
p.x

# meta __init__ -> __new__

1

In [6]:
# cached_property: just like @property, but is executed only once
from functools import cached_property
class Foo:
    a = 0
    @cached_property
    # @property
    def some_property(self):
        self.a += 5
        return self.a 
f = Foo()
# See 5, 10, 15 for @property
# See 5, 5, 5 for @cached_property
print(f.some_property, f.some_property, f.some_property)

5 5 5


In [12]:
# 9 - 23, exec and global variables
a = 1
exec('b = a+1')
# exec can access and return global variables
print(b)

def foo():
    c = 2
    try: 
        # exec are accessing a COPY of the local variables. So d is stored in the copy, 
        # not in the actual local copy
        exec('d = c+1')
        print(d)
    except NameError:
        print("exec doesn't have access to func vars")
        
    # Solution 1 - recommended: pass in a custom dictionary of variables
    loc = {"c" : 2}
    e = 0
    glb = {}
    exec('d = c+1', glb, loc)
    print(loc['d'])
    
    # Solution 2 - not recommended, use local() to access the copy passed into exec
    # Not recommended because each time you call local(), 
    # The copy will be updated by the current local variable values.
    loc = locals()
    exec('e = c+3')
    # Here see 5
    print("after exec, loc copy: ", loc["e"]) 
    locals()
    # Now see 0. 
    print(loc["e"])
            

foo()

2
exec doesn't have access to func vars
3
after exec, loc copy:  5
0


In [1]:
# 9 - 20 - Function overriding with different types 
# 1 Child class to Parent class (upcast) is unconditional
class Foo:
    def __init__(self, a):
        self.a = a 
class Foo2(Foo):
    def __init__(self, a, b):
        super().__init__(a)
        self.b = b
f2 = Foo2(1,2)
print(vars(f2))
print(isinstance(f2, Foo))

# Failed attempt 1
# class -> meta that has MultiDict as clsdict. ->
# MultiDict will store {(type, func_name): method}
class MultiDict(dict):
     def __setitem__(self, key , value ) -> None:
         # We must store string, not tuples because when class gets created, 
         # names in string are compared. But the modified names are 
         # already stored in the class.
         # Failure Reason: therefore, we must store the same key 
         new_key = str((key, 1))
         print(new_key, "val: ", value)
         super().__setitem__(new_key, value)
     def __getitem__(self, key):
        print("get_item, key: ", key)
        return super().__getitem__((key, 1)) 

class Meta(type):
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()

    # def __new__(cls, clsname, bases, clsdict):
    #     return 
class Spam(metaclass = Meta):
    def s(self, a):
        self.a = a
sp = Spam()
try: 
    sp.s(1)
except AttributeError:
    pass




{'a': 1, 'b': 2}
True
get_item, key:  __name__
('__module__', 1) val:  __main__
('__qualname__', 1) val:  Spam
('s', 1) val:  <function Spam.s at 0x7f71de99ddc0>


In [105]:
# 9 - 20 - CROWN JEWEL OF CHAP 9: Function overriding with different types 
# inspect.signature(func)->parameters->name, annotation
import inspect
class TypeMethodRegistry:
    def __init__(self):
        # registry to store different functions under the same name based on their arg list 
        self.registry = {}
        
    def register(self, method):
        '''
        Function takes care of storing functions based on their types
        '''
        # signature has parameters, you have to get their annotations manually
        func_type = []
        # we don't want self, because in signature, the first annotation's type is Meta, 
        # which does not have __eq__ for comparison
        for param in inspect.signature(method).parameters.values():
            if param.name not in ("cls", "self"):
                func_type.append(param.annotation)
        print("registered type: ", func_type)
        # need to hash the list
        self.registry[tuple(func_type)] = method
        
    def __call__(self, *args, **kwds):
        '''
        Function returns the right function based on its types
        self is TypeMethodRegistry, but args will contain the actual instance (Spam)
        '''
        # args will be the values of the arguments directly, including self (which has been bound)
        # Not supporting kwargs, because we need the ORDER of arguments
        arg_types = []
        arg_types = tuple([type(arg) for arg in args[1:]])

        if arg_types in self.registry.keys(): 
            print("Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__: ", args)
            print("So we can pass Spam, and other args to the actual function")
            return self.registry[arg_types](*args)
        else:
            raise AttributeError(f"Function argument annotations {arg_types} are not found in functions with the same name", 
                                 f" {self.registry.keys()}")
        

    def __get__(self, instance, cls):
        '''
        When we call a function of an instance/ class, we call the function's __get__. 
        Then we bound the function to the instance
        In this case, we are using MultiDict, which directly returns TypeMethodRegistry object,
        Which further returns the right function. We need to bind the TypeMethodRegistry to the right 
        instance 
        '''
        
        # cls is always not None
        if instance is None:
            return self
        else:
            from types import MethodType
            print(f"self: {type(self)}, instance: {type(instance)}")
            return MethodType(self, instance)
        
        
class MultiDict(dict):
    '''
    Dict-like object that __prepare__() returns.
    keys are module/ function names
    values are strings (module) or methods
    '''
    # Overriding dictionary method.
    def __setitem__(self, method_name, method_or_str) -> None:
        '''
        We check if we already have the method / module name
        [TAKEAWAY]: You can use try, but in Python dict.__setitem__, you can check by 
        if method in self.
        '''
        if method_name in self:
        # try:
            registry = super().__getitem__(method_name)
            registry.register(method_or_str)
        else:
        # except KeyError as e:
            # because the dict __prepare__ receives things like {"module": "__main__"}        
            if callable(method_or_str):
                method = method_or_str
                registry = TypeMethodRegistry()
                registry.register(method)
            else:
                registry = method_or_str
            super().__setitem__(method_name, registry)
        
    
class Meta(type):
    '''
    Meta's {"func_name": TypeMethodRegistry([method1, method2 ...])} -> Instances (Spam)
    '''
    @classmethod
    def __prepare__(cls, clsname, bases):
        # Have to return a dictionary, with (key, val)
        # In this case, key is function name, val is the method itself 
        return MultiDict()
    
    
class Spam(metaclass = Meta):
    def s(self: Spam):
        print("s1")
    def s(self: Spam, a:int):
        self.a = a
        print("hello")
    def s(self, a:int, b: float):
        self.a = a
        print(self.s.__name__)
sp = Spam()
sp.s()

# So now s() is a class variable, which calls __get__(instance, cls) (Descriptor Protocal for all class variables)
# To get a bound function 
sp.s(1)


registered type:  []
registered type:  [<class 'int'>]
registered type:  [<class 'int'>, <class 'float'>]
self: <class '__main__.TypeMethodRegistry'>, instance: <class '__main__.Spam'>
Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__:  (<__main__.Spam object at 0x7f71dcf21b50>,)
So we can pass Spam, and other args to the actual function
s1
self: <class '__main__.TypeMethodRegistry'>, instance: <class '__main__.Spam'>
Spam is in the args since we have bound this TypeMethodRegistry object to it in __get__:  (<__main__.Spam object at 0x7f71dcf21b50>, 1)
So we can pass Spam, and other args to the actual function
hello


In [34]:
# 9 - 20 - 3 
# Function Overloading using decorator 
# Key Technique: use descriptor as a decorator. 
# See advanced_oop.py -> FooDescriptor for usage
import inspect
class multidictdescriptor:
    def __init__(self, default_func):
        # Step 1 - In descriptor, just need __init__ to store the function
        # Then multidictdescriptor instance replaces the function, as a proxy
        # So later, in the class, we can call f.match
        self.default_func = default_func
        self.registry = {}
         
    def __call__(self, *args, **kwargs):
        # Step 3 - now multidictdescriptor as a proxy, 
        # we need to return the real function
        args_type = [type(arg) for arg in args]
        args_key = tuple(args_type[1:])
        print("call: args key", args_key, " registry: ", self.registry)
        if  args_key in self.registry.keys():
            return self.registry[args_key](*args)
        else:
            return self.default_func(*args, **kwargs)
    
    def match(self, *args):
        # pass types in, like multidictdescriptor.match(int, int)
        def register(func):
            print("function defaults: ", func.__defaults__)
            # TODO: add a check to check if the func signature matches the args
            # since args are passed directly from the decorator,
            # There's no self
            self.registry[args] = func
            # want to return the same multidictdescriptor instance itself back to Spam
            return self
        # the outer layer of decorator always returns the inner decorator function
        return register
        
    def __get__(self, instance, cls):
        # Step 2 - When we call multidictdescriptor instance, 
        # first step is to bind it with Spam instance, 
        # or just return multidictdescriptor instance itself as class instance
        if instance is None:
            return self
        else:
            from types import MethodType
            return MethodType(self, instance)
class Spam:
    @multidictdescriptor
    def f(self, *args, **kwargs):
        raise TypeError(f"No function implemented for args {args}, kwargs {kwargs}")
    
    @f.match(int, int)
    def f(self, a, b):
        print("f1")
    # def f(self, a):
    #     print("f2, ", a)

s = Spam()
s.f(1,2)


None
call: args key (<class 'int'>, <class 'int'>)  registry:  {(<class 'int'>, <class 'int'>): <function Spam.f at 0x7fcdbd6df670>}
f1
